# Wake-on-SNN × BERT Hybrid Chatbot (Colab Demo)このノートブックは、「Wake-on-SNN × BERT Hybrid Chatbot」プロジェクトをGoogle Colab環境で実行するためのデモです。---## 1. 環境構築とファイルの準備プロジェクトに必要なライブラリをインストールし、GitHubリポジトリからファイルをクローンします。

In [ ]:
# 必要なライブラリのインストール!pip install torch transformers scikit-learn# GitHubリポジトリのクローン!git clone https://github.com/alayaproject0-stack/wake-on-snn-chatbot.git# ディレクトリ移動import osos.chdir('wake-on-snn-chatbot')# ファイル構成の確認!ls

---## 2. モジュールの定義と初期化`run_chat.py`のロジックをColabで実行するために、必要なクラス定義をインポートまたは再定義します。**注意:** Colab環境で対話的に実行するため、`run_chat.py`の`input()`ループは次のセクションで別途実装します。

In [ ]:
import jsonimport torchimport numpy as npfrom transformers import pipeline# --- wake_controller.py の内容 ---class WakeController:    def __init__(self, thr):        self.thr = thr    def should_wake(self, confidence):        return confidence < self.thr# --- snn_model.py の内容 (ダミー実装) ---class DummyModel:    def to(self, device): return self    def eval(self): return self    def __call__(self, x):        return torch.tensor([[0.5, 0.5]]), Noneclass DummyVectorizer:    def transform(self, text_list):        return np.array([[0.1, 0.2]])class SNNGate:    def __init__(self, snn_model, vectorizer, device):        self.model = snn_model        self.vec = vectorizer        self.device = device    @torch.no_grad()    def confidence(self, text):        # ダミー実装: テキストの長さに応じてconfidenceを返す        length_score = len(text.split()) / 10.0        confidence_score = max(0.1, 0.8 - length_score)        return confidence_score# --- bert_model.py の内容 (ダミー実装) ---class BertResponder:    def __init__(self):        # 実際にはパイプラインを初期化するが、ここではダミー        pass    def respond(self, text):        # ダミー実装: テキストの長さに応じてラベルを返す        if len(text) > 15:            label = "哲学"        elif len(text) > 8:            label = "技術"        else:            label = "日常"        return f"考えて答えると…この話題は「{label}」ですね。"# --- chatbot.py の内容 ---class WakeChatBot:    def __init__(self, snn_gate, wake_ctrl):        self.snn = snn_gate        self.wake = wake_ctrl        self.bert = BertResponder()        with open("responses.json", "r", encoding="utf-8") as f:            self.light = json.load(f)    def reply(self, text):        conf = self.snn.confidence(text)        if not self.wake.should_wake(conf):            # 軽量応答の検索            for k, v in self.light.items():                if k in text.lower():                    return f"[SNN] {v}"            return "[SNN] なるほど"        return "[BERT] " + self.bert.respond(text)# --- run_chat.py の初期化部分 ---snn_model = DummyModel()vectorizer = DummyVectorizer()# ColabではGPUが利用可能な場合があるため、自動でデバイスを選択device = "cuda" if torch.cuda.is_available() else "cpu"gate = SNNGate(snn_model, vectorizer, device) wake = WakeController(thr=0.434)  # wake_q=0.30由来bot = WakeChatBot(gate, wake)print(f"Chatbot initialized on device: {device}")print("Wake-on-SNN Chatbot 準備完了")

---## 3. チャットボットの実行以下のセルを実行し、`You >` の後に質問を入力してEnterを押してください。

In [ ]:
# Colabでの対話実行用関数def run_interactive_chat(chatbot):    print("--- Wake-on-SNN Chatbot 起動 (exitで終了) ---")    while True:        try:            text = input("You> ")            if text.lower() == "exit":                break            if not text.strip():                continue            print("Bot>", chatbot.reply(text))        except EOFError:            print("\nExiting chat loop.")            break        except KeyboardInterrupt:            print("\nExiting chat loop.")            break# 実行run_interactive_chat(bot)

---## 4. 実行例の確認以下の入力例を試して、SNNとBERTの切り替えを確認してください。| 入力例 | 期待される動作 || :--- | :--- || `hello` | `[SNN] こんにちは😊` (軽量応答) || `ありがとう` | `[SNN] なるほど` (軽量応答のフォールバック) || `人間はなぜ争うの？` | `[BERT] 考えて答えると…この話題は「哲学」ですね。` (BERT起動) || `AIの倫理について教えて` | `[BERT] 考えて答えると…この話題は「技術」ですね。` (BERT起動) |